In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
from zipfile import ZipFile
file_name="/content/drive/My Drive/image-detect.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("finish")

finish


In [3]:
import keras
import pandas as pd
import numpy as np
from keras import layers
from keras.callbacks import History
from matplotlib import pyplot as plt
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,BatchNormalization,Activation,GlobalAveragePooling2D,SeparableConv2D
from keras.models import Sequential,Model,Input
from keras.optimizers import SGD,Adam,RMSprop
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix as cm
from keras.utils.vis_utils import plot_model

In [4]:
labels_path="/content/val/val_annotations.txt"
val_data=pd.read_csv(labels_path,sep="\t",header=None)
val_data.columns=["id","labels","x_max","x_min","y_max","y_min"]
val_data

,id,labels,x_max,x_min,y_max,y_min
0,val_0.JPEG,n07753592,0,18,46,63
1,val_1.JPEG,n02883205,25,48,53,63
2,val_2.JPEG,n04532670,0,7,6,48
3,val_3.JPEG,n03160309,0,34,44,63
4,val_4.JPEG,n07583066,33,13,63,44
...,...,...,...,...,...,...
9995,val_9995.JPEG,n03983396,30,10,37,30
9996,val_9996.JPEG,n04596742,0,7,63,54
9997,val_9997.JPEG,n01698640,0,23,63,57
9998,val_9998.JPEG,n04074963,19,0,59,63


In [5]:
train_datagen=ImageDataGenerator(
       rescale=1./255,
       shear_range=0.2,
       zoom_range=0.2,
       horizontal_flip=True,
       rotation_range=20,
       width_shift_range=0.2,
       height_shift_range=0.2,
       validation_split=0.5)
test_datagen = ImageDataGenerator(
    rescale = 1./255,
    data_format="channels_last",
)
valid_datagen=ImageDataGenerator(rescale=1./255)
train_generator=train_datagen.flow_from_directory(
        directory="train/",
       target_size=(124,124),
       batch_size=14,
       color_mode="rgb",
       class_mode="categorical"
       )
validation_generator=valid_datagen.flow_from_dataframe(
       dataframe=val_data,
       directory="val/images",
       target_size=(124,124),
       batch_size=14,
       x_col="id",
       y_col="labels",
       color_mode="rgb",
       class_mode="categorical"
       )
test_generator = test_datagen.flow_from_directory(
     directory="test/",
    target_size=(124,124),
    color_mode="rgb",
    batch_size=14,
    shuffle=False,
    class_mode=None,
)
img_input=(124,124,3)

Found 90000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.
Found 10000 images belonging to 1 classes.


In [7]:
from keras.applications import Xception
base_model=Xception(weights="imagenet",include_top=False,input_shape=(124,124,3))
x=base_model.output
x=GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x=Activation('relu')(x)
prediction = Dense(200, activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=prediction)

model.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'sgd', metrics=['accuracy'])

In [8]:
from keras.callbacks import History
history=History()
model.fit_generator(
       train_generator,
       epochs=10,callbacks=[history],
       validation_data=validation_generator,

       )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
6429/6429 [==============================] - 587s 90ms/step - loss: 3.3576 - accuracy: 0.3078 - val_loss: 1.3255 - val_accuracy: 0.6810
Epoch 2/10
6429/6429 [==============================] - 576s 90ms/step - loss: 1.5658 - accuracy: 0.6160 - val_loss: 1.2188 - val_accuracy: 0.7038
Epoch 3/10
6429/6429 [==============================] - 577s 90ms/step - loss: 1.3061 - accuracy: 0.6712 - val_loss: 1.1389 - val_accuracy: 0.7209
Epoch 4/10
6429/6429 [==============================] - 597s 93ms/step - loss: 1.1548 - accuracy: 0.7034 - val_loss: 1.1462 - val_accuracy: 0.7241
Epoch 5/10
6429/6429 [==============================] - 573s 89ms/step - loss: 1.0356 - accuracy: 0.7296 - val_loss: 1.0786 - val_accuracy: 0.7345
Epoch 6/10
6429/6429 [==============================] - 576s 90ms/step - loss: 0.9354 - accuracy: 0.7534 - val_loss: 1.1370 - val_accuracy: 0.7251
Epoch 7/10
6429/6429 [==============================] - 575s 89ms/step - loss: 0.8730 - accuracy: 0.7650 - val_loss: 1

In [9]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

pred=model.predict_generator(test_generator,verbose=1)
#prediction file
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)


Saved model to disk


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


715/715 [==============================] - 15s 20ms/step
